In [1]:
from pathlib import Path
import os

from cityslam.videointerface import videointerface, downloader
from cityslam.preprocessing import preprocessing, transitions, create_img_list
from cityslam.mapping import single_video_pipeline
from cityslam.utils import visualization
from natsort import natsorted

# setup necessary paths
base_dir = Path('./demo')

videos_path = base_dir / 'datasets' / 'videos'
images_path = base_dir / 'datasets' / 'images'
queries_path = base_dir / 'datasets' / 'queries'
output_path = base_dir / 'outputs' / 'models'
output_transitions = base_dir / 'outputs' / 'transitions'
output_transitions_cropped = base_dir / 'outputs' / 'transitions_cropped'
image_list_path = base_dir / 'outputs' / 'image_list'

overwrite = False
video_ids = ['gTHMvU3XHBk', 'TZIHy1cZJ-U']
fps = 2

/usr/local/anaconda3/envs/cityslam2/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Download videos
downloader.main(videos_path, video_ids, format="bv", overwrite=overwrite)

In [ ]:

image_folders = preprocessing.main(
    videos_path, images_path, video_ids, overwrite=overwrite, fps=fps)

In [ ]:
path_to_weights = Path('./cityslam/preprocessing')

transitions.main(
    videos_path, video_ids, path_to_weights, output_transitions, output_transitions_cropped, 5*60, 10, fps, 0.5, overwrite_cuts=True)


In [ ]:
create_img_list.create_img_list(output_transitions_cropped, images_path, image_list_path)


In [2]:
# Find all scenes
scene_ids = [str(p.relative_to(image_list_path)).split("_images")[0] for p in natsorted(list(image_list_path.glob("**/*_images.txt")))]
print(f"Total scenes: {len(scene_ids)}")

# Filter out the ones that are already done
#scene_ids = [scene_id for scene_id in scene_ids if next((output_path / scene_id).glob("**/database.db"), None) is None]
scene_ids = [scene_id for scene_id in scene_ids if next((output_path / scene_id).glob("**/images.bin"), None) is None]
print(f"Scenes left: {len(scene_ids)}")

# Run single video pipeline
for scene_id in scene_ids:

    scene_image_list_path = Path(image_list_path) / f"{scene_id}_images.txt"

    reconstruction = single_video_pipeline.main(
        images_path, scene_image_list_path, output_path, video_id=scene_id, window_size=6, num_loc=6, pairing='sequential+retrieval', run_reconstruction=True, overwrite=False)


[2022/06/10 13:53:15 hloc.utils.parsers INFO] Imported 41 images from part00_images.txt
[2022/06/10 13:53:15 hloc INFO] Extracting local features with configuration:
{'model': {'name': 'netvlad'},
 'output': 'global-feats-netvlad',
 'preprocessing': {'resize_max': 1024}}
[2022/06/10 13:53:15 hloc INFO] Skipping the extraction.
[2022/06/10 13:53:15 hloc INFO] Extracting image pairs from a retrieval database.


Total scenes: 2
Scenes left: 1
getting images...
num images : 41


[2022/06/10 13:53:15 hloc INFO] Found 54 pairs.
[2022/06/10 13:53:15 hloc INFO] Found 346 pairs.
[2022/06/10 13:53:15 hloc INFO] Extracting local features with configuration:
{'model': {'max_keypoints': 4096, 'name': 'superpoint', 'nms_radius': 3},
 'output': 'feats-superpoint-n4096-r1024',
 'preprocessing': {'grayscale': True, 'resize_max': 1024}}
[2022/06/10 13:53:15 hloc INFO] Skipping the extraction.
[2022/06/10 13:53:15 hloc INFO] Matching local features with configuration:
{'model': {'name': 'superglue',
           'sinkhorn_iterations': 50,
           'weights': 'outdoor'},
 'output': 'matches-superglue'}
[2022/06/10 13:53:15 hloc INFO] Skipping the matching.
[2022/06/10 13:53:15 hloc INFO] Creating an empty database...
[2022/06/10 13:53:15 hloc INFO] Importing images into the database...
[2022/06/10 13:53:16 hloc INFO] Importing features into the database...
100%|██████████| 41/41 [00:00<00:00, 713.17it/s]
[2022/06/10 13:53:16 hloc INFO] Importing matches into the database...
1

In [ ]:
"""
* run find_model_pairs for the two videos
* run score_pipeline
* run graph-pipeline
"""